# 02 - Creation d'un dataset des données nettoyées de metrics (après fractionnement)

Ce notebook génère :

- 1 fichier csv "merge_clean_metrics_dataset.csv"

Etapes de nettoyage :

- Suppression des colonnes n'ayant que des valeurs nulles

- Suppression des colonnes avec informations redondantes (identification=message_events) ou inutiles (id de message)

- Conversion des types de colonnes avec le type de valeurs

- Remplacement des valeurs nulles

- Encodage des codes d'identification en chaine de caractères (maj du metrics_events_dict.json) et de la criticité

# A. Imports

## Librairies

In [1]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path

## Données

In [2]:
# source path to raw metrics dataset
source_csv = '../data/metrics/raw_merge_metrics_dataset.csv'
# target path to save metrics dictionnaire
save_json ='../data/metrics/metrics_events_dict.json'
# target path to save merge raw metrics dataset
save_csv = '../data/metrics/clean_merge_metrics_dataset.csv'
encoded_save_csv = '../data/metrics/encoded_clean_merge_metrics_dataset.csv'

# B. Dataframe

## a) Import dataset

In [3]:
# création d'un dataframe à partir du csv de données
df = pd.read_csv(Path(source_csv), index_col=0)
# réindexation à 0
df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761371 entries, 0 to 761370
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         761371 non-null  int64  
 1   source_events              30316 non-null   object 
 2   message_events             30295 non-null   object 
 3   timestamp_events           30316 non-null   object 
 4   criticality_events         30316 non-null   object 
 5   identification_events      30316 non-null   object 
 6   sn_modules                 0 non-null       float64
 7   name_modules               761371 non-null  object 
 8   type_modules               761371 non-null  object 
 9   generation_modules         505644 non-null  object 
 10  name_counters_modules      761371 non-null  object 
 11  value_counters_modules     761371 non-null  int64  
 12  name_connected_operators   761371 non-null  object 
 13  level_connected_operators  76

## b) Selection des colonnes

In [4]:
# suppression des colonnes ne contenant que des valeurs nulles
df = df.dropna(axis=1, how='all')

In [5]:
# on supprime les colonnes doublons (message=identification)
df = df.drop(['id', 'message_events'], axis=1)

In [6]:
# on converti les float en entier 64
df.varnishLevelsTargetvolume = pd.to_numeric(df.varnishLevelsTargetvolume).astype('int64')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761371 entries, 0 to 761370
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   source_events              30316 non-null   object
 1   timestamp_events           30316 non-null   object
 2   criticality_events         30316 non-null   object
 3   identification_events      30316 non-null   object
 4   name_modules               761371 non-null  object
 5   type_modules               761371 non-null  object
 6   generation_modules         505644 non-null  object
 7   name_counters_modules      761371 non-null  object
 8   value_counters_modules     761371 non-null  int64 
 9   name_connected_operators   761371 non-null  object
 10  level_connected_operators  761371 non-null  object
 11  status                     761371 non-null  object
 12  created_at                 761371 non-null  object
 13  varnishLevelsTargetvolume  761371 non-null  

### 1) Colonne 'timestamp'

In [8]:
# on renomme la colonne timestamp_events
df = df.rename(columns={'timestamp_events':'timestamp'})
# on remplace des valeurs maquantes de timestamp par celle de created_at
df.timestamp = df.timestamp.fillna(df['created_at'])
# on converti les valeur en datetim
df.timestamp = pd.to_datetime(df.timestamp, utc=True)
# on supprime la colonne doublon (created_at=tiemstamp)
df = df.drop(['created_at'], axis=1)

### 2) Colonne 'identification'

In [9]:
df.identification_events.unique()

array(['407', '330', '332', '376', '333', '391', '480', '430', nan, '313',
       '331', '384', '358', '377', '356', '334', '454', '355', '357',
       '381', 'iFoil communication error', '320', '383', '354', '352',
       'Kernel_Error', '417', '466', '445', '405',
       'RCB communication error', '311', '321', '386', '446', '350',
       '372', '314', '385', '440', '345', '346', '0', '380', '371', '388',
       '325', '447', '406', '453', '344', '343', '451', '460', '408',
       '382', '328', '416', '387', '389', '418'], dtype=object)

In [10]:
# on remplace les valeurs nulles par des 0
df.identification_events = df.identification_events.replace(np.nan, 0)

In [11]:
# on encode les valeurs du type 'str' avec un code
events_id = []
str_code_dict = {}
str_code = 1000
for id in list(df['identification_events'].unique()) :
    try:
        events_id.append(int(id))
    except ValueError:
        str_code_dict[id] = str_code
        events_id.append(str_code)
        str_code += 1
str_code_dict

{'iFoil communication error': 1000,
 'Kernel_Error': 1001,
 'RCB communication error': 1002}

In [12]:
# on sauvegarde l'encodage dans metrics_events_dict
inv_str_code_dict = {v: k for k, v in str_code_dict.items()}
with open(file=Path(save_json), mode="r+", encoding='utf-8') as jsonFile:
    data = json.load(jsonFile)
    data['identification encoded'] = inv_str_code_dict
    jsonFile.seek(0)
    json.dump(data, jsonFile, indent=4, ensure_ascii=False)
    jsonFile.close()

In [13]:
# on remplace dans le dataframe les valeurs du type 'str' avec un code
df.identification_events = df.identification_events.replace(str_code_dict)
# on converti toutes les valeurs en entier
df.identification_events = pd.to_numeric(df.identification_events).astype('int64')

In [14]:
df.identification_events.unique()

array([ 407,  330,  332,  376,  333,  391,  480,  430,    0,  313,  331,
        384,  358,  377,  356,  334,  454,  355,  357,  381, 1000,  320,
        383,  354,  352, 1001,  417,  466,  445,  405, 1002,  311,  321,
        386,  446,  350,  372,  314,  385,  440,  345,  346,  380,  371,
        388,  325,  447,  406,  453,  344,  343,  451,  460,  408,  382,
        328,  416,  387,  389,  418], dtype=int64)

### 3) Encodage des labels 'criticality'

In [15]:
encoded_df = df.copy()

In [16]:
# on remplace dans le dataframe les valeurs du type 'str' avec un code
encoded_df.criticality_events = encoded_df.criticality_events.fillna("UNDEFINED")
criticality = {'UNDEFINED': 0, 'INFO': 1, 'WARNING': 2, 'ERROR':3}
encoded_df.criticality_events.replace(criticality, inplace=True)
encoded_df.criticality_events = pd.to_numeric(encoded_df.criticality_events).astype('int64')

In [17]:
# on sauvegarde l'encodage dans metrics_events_dict
inv_criticality = {v: k for k, v in criticality.items()}
with open(file=Path(save_json), mode="r+", encoding='utf-8') as jsonFile:
    data = json.load(jsonFile)
    data['criticality encoded'] = inv_criticality
    jsonFile.seek(0)
    json.dump(data, jsonFile, indent=4, ensure_ascii=False)
    jsonFile.close()

## c) Output csv

In [18]:
# sauvegarde du dataframe avant encodage
df.to_csv(path_or_buf=Path(save_csv))

In [19]:
df.head(3)

,source_events,timestamp,criticality_events,identification_events,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,status,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-11-03 08:43:16.808000+00:00,INFO,407,iFoil L,iFoil,Gen. 2,Total Pages Counter,31185,User,Operator,WARNING,1386,18000
1,iFoil,2022-11-03 08:43:16.808000+00:00,INFO,407,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,79566,User,Operator,WARNING,1386,18000
2,PLC,2022-11-03 08:43:16.746000+00:00,WARNING,407,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,308536,User,Operator,WARNING,1386,18000


In [20]:
# sauvegarde du dataframe après encodage
encoded_df.to_csv(path_or_buf=Path(encoded_save_csv))